Based on these posts: 
* https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
* https://github.com/keisukeirie/Amazon_review_helpfulness_prediction
* https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
* https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

In [ ]:
!pip install -q boto3
!pip install -q xgboost

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
#from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#from sklearn.linear_model import SGDClassifier

classifier = Pipeline([
    ('features', FeatureUnion([
        ('body', Pipeline([
            ('body_text_selector', TextSelector('review_body')),
            ('tfidf_vectorizer', TfidfVectorizer(tokenizer=Tokenizer, stop_words="english",
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
#        ('star_rating', Pipeline([
#            ('star_rating_selector', NumberSelector('star_rating')),
#            ('standard_scaler', StandardScaler()),
#        ])),
    ])),
#    ('classifier', SGDClassifier(learning_rate='optimal'))
    ('classifier', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('classifier', RandomForestClassifier()),
    ])

Fit the model

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

preds = classifier.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, preds))
print('Precision: ', precision_score(y_test, preds, average=None))

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
df_cm = confusion_matrix(y_test, preds)
df_cm

In [ ]:
# TODO:  Explain the classifier

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#plt.figure(figsize = (10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

# TODO:  Add labels to each quadrant (False, True / False, True)

plt.show()

In [ ]:
predictions, raw_outputs = bert_model.predict(["""Very funny. A typical mid 50's comedy."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = bert_model.predict(["""That movie was absolutely awful."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))